<a href="https://colab.research.google.com/github/mahad-asim76/Final-Year-Project/blob/main/API_For_Bitcoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import math
import yfinance as yf
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default="colab"
from datetime import date, timedelta
today = date.today()
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Model = keras.models.load_model('/content/drive/MyDrive/LSTM Models/Bitcoin_Model.h5')

In [ ]:
d1 = date.today() + timedelta(days=1)
d1 = d1.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=1521)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

In [ ]:
df = yf.download('BTC-USD', start=start_date, end=end_date, progress=False)
df.reset_index(inplace=True)

In [ ]:
data = df.filter(['Close'])
dataset = data.values
training_data_len = math.ceil(len(dataset) * .7)
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

In [ ]:
test_data = scaled_data[training_data_len - 60 : , :]
X_test = []
y_test = dataset[training_data_len : , :]
for i in range(60, len(test_data)):
  X_test.append(test_data[i-60 : i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
x_input = np.array(test_data[-60:]).reshape(1,-1)
new_price = np.array(df['Close'][len(df)-16:len(df)-1]).reshape(-1,1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [ ]:
from numpy import array
lst_output = []
n_steps = 60
i = 0
while i < 60:
    if len(temp_input) > 60:
        x_input = array(temp_input[1:])
        x_input = x_input.reshape(1, -1)
        x_input = x_input.reshape((1, n_steps, 1))
        yhat = Model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        temp_input = temp_input[1:]
        lst_output.extend(yhat.tolist())
        i += 1
    else:
        x_input = x_input.reshape((1, n_steps, 1))
        yhat = Model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        lst_output.extend(yhat.tolist())
        i += 1

In [ ]:
pred_price = scaler.inverse_transform(lst_output)
price = np.array(pred_price)
datestamp = []
price = np.concatenate((new_price, price[2:18]), axis=0)
df_forecast = pd.DataFrame(price[2:18], columns=['Forecast'])
for i in range(len(price)):
  date = date.today() + timedelta(days = i)
  datestamp.append(date)

datestamp = np.array(datestamp, dtype='datetime64')
new_date = np.array(df['Date'][len(df)-16:len(df)-1])
new_stamp = np.concatenate((new_date, datestamp[2:18]), axis=0)
df_date = pd.DataFrame(datestamp[2:18], columns = ["Date"])
forecast = pd.concat([df_date, df_forecast], axis=1)

In [ ]:
trace1 = go.Scatter(
    x = forecast['Date'],
    y =  forecast['Forecast'],
    mode = 'lines',
    name = 'Forecast'
)


Data = [trace1]
fig = dict(data=Data)
iplot(fig, filename = "Time Series with Rangeslider")

In [ ]:
forecast.to_csv('/content/drive/My Drive/Data/BTC.csv', index=False)